In [26]:
import os

In [27]:
%pwd

'c:\\Users\\rahul\\Facial_review_system'

In [28]:
os.chdir('../')

In [29]:
%pwd

'c:\\Users\\rahul'

In [30]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    base_model_path: Path
    params_image_size: list
    params_classes: int
    

In [31]:
from Facial_Review_System.constants import *
from Facial_Review_System.utils.common import read_yaml, create_directories

In [32]:
#this class for base model configuration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            base_model_path=Path(config.base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [33]:
import os
import zipfile
import gdown
from Facial_Review_System import logger
from Facial_Review_System.utils.common import get_size

In [34]:
#this class is for downloading model 
class ModelDownload:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            os.makedirs(self.config.root_dir, exist_ok=True)  # Updated directory path
            logger.info(f"Downloading data from {dataset_url} into file {self.config.local_data_file}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, self.config.local_data_file)  # Save to local_data_file

            logger.info(f"Downloaded data from {dataset_url} into file {self.config.local_data_file}")

        except Exception as e:
            raise e

In [35]:
try:
    config = ConfigurationManager()
    model_config = config.get_model_config() 
    vgg_model = ModelDownload(config = model_config)
    zip_download_dir = model_config.local_data_file 
    vgg_model.download_file()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'